In [1]:
import sys
from pathlib import Path

sys.path.append(str(Path().resolve().parent))

In [2]:
from src.data.load_data import load_rr_data, load_actigraph_data

user_path = Path("../data/raw/user_1/")

rr_df = load_rr_data(user_path)
act_df = load_actigraph_data(user_path)

print(rr_df.head())
print(act_df.head())

   Unnamed: 0  ibi_s  day      time           timestamp
0           1  0.258    1  10:10:17 2023-01-01 10:10:17
1           2  0.319    1  10:10:18 2023-01-01 10:10:18
2           3  0.266    1  10:10:18 2023-01-01 10:10:18
3           4  0.401    1  10:10:18 2023-01-01 10:10:18
4           5  1.089    1  10:10:19 2023-01-01 10:10:19
   Unnamed: 0  Axis1  Axis2  Axis3  Steps     HR  Inclinometer Off  \
0          82      0      0      0      0   68.0                 0   
1          83     11      4      7      1   68.0                 0   
2          84      0     21     10      0   68.0                 0   
3          85      0      1     24      0   68.0                 0   
4          86     34     14     63      1  154.0                 0   

   Inclinometer Standing  Inclinometer Sitting  Inclinometer Lying  \
0                      1                     0                   0   
1                      1                     0                   0   
2                      0         

In [3]:
from src.data.load_data import load_rr_data
from pathlib import Path

rr_df = load_rr_data(Path("../data/raw/user_1"))

rr_df["bpm"] = 60 / rr_df["ibi_s"]

# On met l'index sur les timestamps
rr_df = rr_df.set_index("timestamp")

# On supprime les doublons éventuels de timestamp
rr_df = rr_df[~rr_df.index.duplicated(keep="first")]

# Interpolation linéaire à 1Hz (1 valeur par seconde)
rr_interp = rr_df["bpm"].resample("1s").mean().interpolate()

# Reset index pour repasser timestamp en colonne
rr_interp = rr_interp.reset_index()

In [4]:
from src.data.preprocess import prepare_user_data
prepare_user_data("user_1", raw_base="../data/raw", out_base="../data/processed")

✅ Données fusionnées sauvegardées : ../data/processed/user_1.csv


In [5]:
import pandas as pd

df = pd.read_csv("../data/raw/user_9/RR.csv")
print(df["day"].unique())
print(df.head())

[  1 -29]
   Unnamed: 0  ibi_s  day      time
0           1  0.595    1  10:06:32
1           2  0.341    1  10:06:32
2           3  1.281    1  10:06:34
3           4  0.375    1  10:06:34
4           5  3.126    1  10:06:37


In [6]:
from src.data.preprocess import window_features
import pandas as pd

df = pd.read_csv("../data/processed/user_1.csv")
df["timestamp"] = pd.to_datetime(df["timestamp"])

features = window_features(df, user_id="user_1", freq="60s")
features.head()

,timestamp,hr_mean,hr_std,vector magnitude_mean,vector magnitude_std,steps_sum,user_id
0,2023-01-01 10:10:00,133.818182,52.723464,60.260000,54.633228,4,user_1
1,2023-01-01 10:11:00,96.357143,49.780452,95.919286,66.822908,10,user_1
2,2023-01-01 10:12:00,110.244444,19.195354,171.609333,139.523331,24,user_1
3,2023-01-01 10:13:00,89.705882,3.864164,51.317059,75.549314,9,user_1
4,2023-01-01 10:14:00,93.360000,3.075645,123.001400,93.779791,19,user_1


In [7]:
import pandas as pd
pd.read_csv("../data/raw/user_1/sleep.csv").columns

Index(['Unnamed: 0', 'In Bed Date', 'In Bed Time', 'Out Bed Date',
       'Out Bed Time', 'Onset Date', 'Onset Time', 'Latency', 'Efficiency',
       'Total Minutes in Bed', 'Total Sleep Time (TST)',
       'Wake After Sleep Onset (WASO)', 'Number of Awakenings',
       'Average Awakening Length', 'Movement Index', 'Fragmentation Index',
       'Sleep Fragmentation Index'],
      dtype='object')